In [163]:
# import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')

from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import word_tokenize #Used to extract words from documents
# from nltk.stem import WordNetLemmatizer #Used to lemmatize words
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans

import sys
from time import time

import pandas as pd
import numpy as np

import spacy
nlp = spacy.load('en_core_web_trf') 

In [164]:
categories = [
    "alt.atheism",
    "comp.graphics",
    "comp.sys.ibm.pc.hardware",
    # "comp.sys.mac.hardware",
    # "comp.windows.x",
    # "misc.forsale",
    # "rec.autos",
    # "rec.motorcycles",
    # "rec.sport.baseball",
    # "rec.sport.hockey",
    # "sci.crypt",
    # "sci.electronics",
    # "sci.med",
    # "sci.space",
    # "soc.religion.christian",
    # "talk.politics.guns",
    # "talk.politics.mideast",
    # "talk.politics.misc",
    # "talk.religion.misc",
    # "comp.os.ms-windows.misc",
]
data = fetch_20newsgroups(categories=categories, remove=('headers', 'footers', 'quotes'))
y_true = data.target
# y_true = newsgroups.targetdata = fetch_20newsgroups(subset='train', categories=['alt.atheism', 'sci.med', 'sci.space'], 
#                              shuffle=False, remove=('headers', 'footers', 'quotes'))

# df = pd.DataFrame(data=data.data)

# len(df)
# print(type(df))
# # df.data[0]
# df.head(3)

In [165]:
df = pd.DataFrame()
df['text'] = data.data
df.head()

,text
0,Cyrix have released a 386 pin-conpatible 486 c...
1,"Andrew,\nYou can get the heat sinks at Digi-Ke..."
2,I saw this subject and all I could think of wa...
3,"\n\n\n\nNo, Mathew is proposing a public defen..."
4,\n\nI thought this was a neat feature until I ...


In [166]:
df['target'] = [categories[x] for x in y_true]
df.head()

,text,target
0,Cyrix have released a 386 pin-conpatible 486 c...,comp.sys.ibm.pc.hardware
1,"Andrew,\nYou can get the heat sinks at Digi-Ke...",comp.sys.ibm.pc.hardware
2,I saw this subject and all I could think of wa...,comp.graphics
3,"\n\n\n\nNo, Mathew is proposing a public defen...",alt.atheism
4,\n\nI thought this was a neat feature until I ...,comp.sys.ibm.pc.hardware


In [167]:
df['text'][1]

'Andrew,\nYou can get the heat sinks at Digi-Key 1-800-344-4539 part #HS157-ND\n$4.10  size 1.89"L x 1.89"W x .600"H  comes with clips to install it.\nBut if it was me I would get a $12.99 small fan from Radio Shack\nand install it where it could just blow at the cpu instead...Sam'

In [168]:
labels = df.target
true_k = len(np.unique(labels)) ## This should be 3 in this example
print(true_k)

3


In [169]:
df_a = df[df['target'] == 'alt.atheism'][:3]
df_b = df[df['target'] == 'comp.graphics'][:3]
df_c = df[df['target'] == 'comp.sys.ibm.pc.hardware'][:3]
df = pd.concat([df_a, df_b, df_c])
df.reset_index(drop=True, inplace=True)
df

,text,target
0,"\n\n\n\nNo, Mathew is proposing a public defen...",alt.atheism
1,"\nHumans have ""gone somewhat beyond"" what, exa...",alt.atheism
2,"\n\nFor the last time, Bobby. Lack of belief i...",alt.atheism
3,I saw this subject and all I could think of wa...,comp.graphics
4,Does anyone know where I can FTP MPEG for DOS ...,comp.graphics
5,"\nIn the UIBM PC world, how much of a ""standa...",comp.graphics
6,Cyrix have released a 386 pin-conpatible 486 c...,comp.sys.ibm.pc.hardware
7,"Andrew,\nYou can get the heat sinks at Digi-Ke...",comp.sys.ibm.pc.hardware
8,\n\nI thought this was a neat feature until I ...,comp.sys.ibm.pc.hardware


In [170]:
# Токенизация
def number_normalizer(tokens):
    """Map all numeric tokens to a placeholder.

    For many applications, tokens that begin with a number are not directly
    useful, but the fact that such a token exists can be relevant.  By applying
    this form of dimensionality reduction, some methods may perform better.
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

for i in range(len(df['text'])):
    tokenized_text = word_tokenize(df['text'][i])
    # tokenized_text = number_normalizer(tokenized_text)
    tokenized_text = [' '.join([w for w in tokenized_text if w.isalpha() and len(w)>1])]
    df['text'][i] = ' '.join([f.lower() for f in tokenized_text])
    
df

,text,target
0,no mathew is proposing public defence mechanis...,alt.atheism
1,humans have gone somewhat beyond what exactly ...,alt.atheism
2,for the last time bobby lack of belief in your...,alt.atheism
3,saw this subject and all could think of was pa...,comp.graphics
4,does anyone know where can ftp mpeg for dos fr...,comp.graphics
5,in the pc world how much of standard has vesa ...,comp.graphics
6,cyrix have released clone designed to upgrade ...,comp.sys.ibm.pc.hardware
7,andrew you can get the heat sinks at part size...,comp.sys.ibm.pc.hardware
8,thought this was neat feature until noticed th...,comp.sys.ibm.pc.hardware


In [171]:
def remove_names(text):
    doc = nlp(text)
    newString = text
    for e in reversed(doc.ents):
        if e.label_ == "PERSON": # Only if the entity is a PERSON
            newString = newString[:e.start_char] + newString[e.start_char + len(e.text):]
    return newString


for i in range(len(df['text'])):
    df['text'][i] = remove_names(df['text'][i])
df

,text,target
0,no is proposing public defence mechanism not ...,alt.atheism
1,humans have gone somewhat beyond what exactly ...,alt.atheism
2,for the last time lack of belief in your god ...,alt.atheism
3,saw this subject and all could think of was pa...,comp.graphics
4,does anyone know where can ftp mpeg for dos fr...,comp.graphics
5,in the pc world how much of standard has vesa ...,comp.graphics
6,cyrix have released clone designed to upgrade ...,comp.sys.ibm.pc.hardware
7,you can get the heat sinks at part size comes...,comp.sys.ibm.pc.hardware
8,thought this was neat feature until noticed th...,comp.sys.ibm.pc.hardware


In [172]:
df['text'][7]

' you can get the heat sinks at part size comes with clips to install it but if it was me would get small fan from radio shack and install it where it could just blow at the cpu instead '

In [173]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

for i in range(len(df['text'])):
    df['text'][i] = [word for word in df['text'][i].split() if word not in stop_words]
    df['text'][i] = " ".join(str(x) for x in df['text'][i])

df.to_csv('preprocessed_docs.csv', index=False)

df

,text,target
0,proposing public defence mechanism treating el...,alt.atheism
1,humans gone somewhat beyond exactly one thread...,alt.atheism
2,last time lack belief god imply atheism moslem...,alt.atheism
3,saw subject could think parade wrigley field c...,comp.graphics
4,anyone know ftp mpeg dos thanks help advance e...,comp.graphics
5,pc world much standard vesa become svga graphi...,comp.graphics
6,cyrix released clone designed upgrade old chip...,comp.sys.ibm.pc.hardware
7,get heat sinks part size comes clips install w...,comp.sys.ibm.pc.hardware
8,thought neat feature noticed image scanning fr...,comp.sys.ibm.pc.hardware


In [179]:
for i in range(9):
    for j in range(9):
        if i!= j:
            si = df['text'][i]
            sj = df['text'][j]
            print(i, j)
            print(si)
            print(sj)
            si = set(si.split())
            sj = set(sj.split())
            print(si.intersection(sj))
            print('-'*10)
            print()

0 0
proposing public defence mechanism treating electronic device impropriety wearer saying next step beyond propose permanent bugging potential criminals may surface sound like bad thing defines potential criminal government day decides member opposition party makes potential criminal openly defying government becomes lethal practice conducive free society saying implanting electronic surveillance devices upon people impropriety upon person regardless type crime chance recidivism basically see criminal justice system punishment offender possibly therefore deterrant future offenders sees probably means rehabilitation offender cynical okay
proposing public defence mechanism treating electronic device impropriety wearer saying next step beyond propose permanent bugging potential criminals may surface sound like bad thing defines potential criminal government day decides member opposition party makes potential criminal openly defying government becomes lethal practice conducive free socie

AttributeError: 'set' object has no attribute 'split'

In [175]:
vectorizer = TfidfVectorizer(strip_accents='unicode') ## Corpus is in English
X = vectorizer.fit_transform(df['text'])
X

<9x258 sparse matrix of type '<class 'numpy.float64'>'
	with 281 stored elements in Compressed Sparse Row format>

In [176]:
print(X.shape)
print(type(X))
for i in range(9):
    print(*X[i, :].toarray())

(9, 258)
<class 'scipy.sparse._csr.csr_matrix'>
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.09945146 0.09945146 0.
 0.09945146 0.         0.         0.         0.08399828 0.
 0.         0.         0.09945146 0.         0.         0.
 0.         0.09945146 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.09945146 0.         0.
 0.         0.         0.         0.09945146 0.29835437 0.09945146
 0.         0.09945146 0.         0.09945146 0.09945146 0.
 0.09945146 0.09945146 0.         0.09945146 0.         0.
 0.09945146 0.09945146 0.09945146 0.         0.         0.
 0.         0.19890292 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.09945146 0.         0.         0.
 0.09945146 0.         0.         0.         0.         0.


In [177]:
df2 = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out())
df2

,ability,able,actions,adhering,adjust,advance,allah,also,although,animals,...,use,vesa,vlb,wearer,well,work,world,would,wrigley,wrong
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.099451,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.155343,0.233015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077672,...,0.065603,0.00000,0.000000,0.000000,0.057040,0.000000,0.000000,0.000000,0.000000,0.310687
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.338226,0.000000,0.169113,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.205501,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.299609,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.185377,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.55613,0.000000,0.000000,0.136135,0.000000,0.185377,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.181391,0.000000,0.000000,...,0.000000,0.00000,0.181391,0.000000,0.133208,0.181391,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.218893,0.000000,0.000000
8,0.128548,0.000000,0.000000,0.000000,0.128548,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.217147,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [178]:
res = pd.DataFrame(df2)
res.to_csv('res.csv', index=False)
res

,ability,able,actions,adhering,adjust,advance,allah,also,although,animals,...,use,vesa,vlb,wearer,well,work,world,would,wrigley,wrong
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.099451,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.155343,0.233015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077672,...,0.065603,0.00000,0.000000,0.000000,0.057040,0.000000,0.000000,0.000000,0.000000,0.310687
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.338226,0.000000,0.169113,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.205501,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.299609,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.185377,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.55613,0.000000,0.000000,0.136135,0.000000,0.185377,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.181391,0.000000,0.000000,...,0.000000,0.00000,0.181391,0.000000,0.133208,0.181391,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.218893,0.000000,0.000000
8,0.128548,0.000000,0.000000,0.000000,0.128548,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.217147,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
